# Detecting Spam

### CSEC620 - Project 4 - Mehul Sen



### Import Required Libraries

In [9]:
# Import numpy for data handling, and basic math calculations
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras

from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import KNeighborsClassifier
from keras import layers
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

### Preprocessing the Dataset
Dataset gets loaded from `SMSSpamCollection` file. The letters are then lowercased, and all numbers are removed. The dataset is split into train and test. Additionally, the dataset is then tokenized to update the vocabulary based on the list of SMS texts. After being tokenized, the messages are converted into a sequence of integers. Lastly the train and test sequences are padded such that they are equal to the maximum length of the texts.

In [10]:
# Load the dataset
data = pd.read_csv('./Dataset/SMSSpamCollection', sep='\t', names=["label", "message"])

# Preprocess the dataset
data['label'] = data['label'].map({'spam': 1, 'ham': 0})
# Lowercase all words
data['message'] = data['message'].apply(lambda x: x.lower())

# Remove numbers
data['message'] = data['message'].str.replace('\d+', '')
y_true = data['label'].values

# Split the dataset into training and testing sets
train_data, test_data = train_test_split(data, test_size=0.2)

# Tokenize the messages
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_data['message'])

# Convert the messages to sequences
train_sequences = tokenizer.texts_to_sequences(train_data['message'])
test_sequences = tokenizer.texts_to_sequences(test_data['message'])

# Pad the sequences so they are all same length
max_length = max([len(seq) for seq in train_sequences])
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding='post')

C:\Users\MSI\AppData\Local\Temp\ipykernel_15624\2422552367.py:10: FutureWarning: The default value of regex will change from True to False in a future version.
  data['message'] = data['message'].str.replace('\d+', '')


### Term Document Matrix

The dataset is vectorized using the term frequency and the inverse document frequency methods. The `TfidfVectorizer` is initialized with `stop_words='english'`, which means it will ignore common English words like 'is', 'the', 'and', etc. that do not contain important meaning and are not useful for the model. The `fit_transform` method learns the vocabulary in the given text data and returns a matrix `M` where each row corresponds to a document and each column corresponds to a word in the vocabulary, and the value in each cell is the TF-IDF score of the word in the corresponding document.

In [11]:
# Get the vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Vocabulary size: %d' % vocab_size)

vectorizer = TfidfVectorizer(stop_words='english')
M = vectorizer.fit_transform(data['message'])
print("Shape of Matrix: ", M.shape)

Vocabulary size: 7178
Shape of Matrix:  (5572, 7553)


### K-Means Algorithm
This algorithm is a type of unsupervised machine learning used to identify clusters of data.
The `KMeans` class has three main components:
1. The `__init__` method: This is the constructor of the class. It initializes the class with the number of clusters `K` and the maximum number of iterations `max_iters` for the algorithm to converge.
2. The `fit` method: This method is where the KMeans algorithm is implemented. It starts by randomly initializing the centroids of the clusters. Then, it enters a loop that will run for a maximum number of iterations. In each iteration, it assigns each data point in `X` to the closest centroid, and then updates each centroid to be the mean of the points assigned to it.
3. The `predict` method: This method is used to assign each data point in `X` to the closest centroid. It calculates the Euclidean distance from each point to each centroid, and then returns the index of the closest centroid for each point.

This code block also includes a test of the KMeans algorithm. It creates a `KMeans` object with 2 clusters and a maximum of 100 iterations, fits the model to the data `M.toarray()`, and then predicts the cluster assignments for the same data. It takes around 3 minutes to run completely.

Last Performance Metrics:
```
KMeans Clustering Results: 
Precision: 0.4356211865639221
Recall: 0.48071692642764496
F1 Score: 0.4558170408516287
Accuracy: 0.8305814788226848
```

In [18]:
# Define the KMeans Algorithm
class KMeans:
    def __init__(self, K=2, max_iters=150):
        # Number of clusters 
        self.K = K  
        # Maximum number of iterations for convergence
        self.max_iters = max_iters

    def fit(self, X):
        # Intialize K random centroids
        self.centroids = X[np.random.choice(range(X.shape[0]), size=self.K, replace=False)]
        for _ in range(self.max_iters):
            # Assign points to the nearest centroid
            labels = self.predict(X)
            # Recompute centroids as cluster means
            for i in range(self.K):
                self.centroids[i] = np.mean(X[labels == i], axis=0)

    def predict(self, X):
        # Calculate the distance of each point to each centroid
        distances = np.sqrt(((X[:, np.newaxis] - self.centroids) ** 2).sum(axis=2))
        # Assign each point to the closest centroid
        return np.argmin(distances, axis=1)
    
# Test the KMeans Algorithm
num_clusters = 2
k = KMeans(K=num_clusters, max_iters=150)
k.fit(M.toarray())
y_pred = k.predict(M.toarray())

# Calculate the performance metrics
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
accuracy = accuracy_score(y_true, y_pred)

# Print the performance metrics
print("KMeans Clustering Results: ")
print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1 Score: {f1}')
print(f'Accuracy: {accuracy}')


KMeans Clustering Results: 
Precision: 0.4356211865639221
Recall: 0.48071692642764496
F1 Score: 0.4558170408516287
Accuracy: 0.8305814788226848


### K Nearest Neighbor Algorithm

It is a simple and widely used algorithm for classification and regression tasks in machine learning. It works based on the principle that similar things exist in close proximity. In classification tasks, when given a new, unknown data point, kNN identifies its category by analyzing the categories of the 'k' nearest data points in the feature space. The majority class among these neighbors determines the classification of the new point. This class is implemented using the `sklearn.neighbors` module in the Scikit library. To match the results of the paper, this algorithm uses K=1 (1NN), however this can be modified to consider more nearest neighbours.

Last Performance Metrics:
```
Precision: 1.0
Recall: 0.6358381502890174
F1 score: 0.7773851590106007
Accuracy: 0.9486133768352365
```

In [20]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(M.toarray(), data['label'], test_size=0.22)

# Number of Neighbors to use
neighbors = 1

# Train the K Nearest Neighbor on the training data
knn = KNeighborsClassifier(n_neighbors=neighbors, weights='distance')
knn.fit(X_train, y_train)

# Predict the labels for the testing data
y_pred = knn.predict(X_test)

# Calculate the performance metrics
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

# Print the performance metrics
print("KNN Results: ")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 score: {f1}")
print(f"Accuracy: {accuracy}")

KNN Results: 
Precision: 1.0
Recall: 0.6358381502890174
F1 score: 0.7773851590106007
Accuracy: 0.9486133768352365


### Transformer Algorithm

Transformers are a type of model architecture introduced in the paper "Attention is All You Need" by Vaswani et al. They are particularly effective for many Natural Language Processing (NLP) tasks. 
 
The code is adapted from ```https://keras.io/examples/nlp/text_classification_with_transformer/``` and defines two classes: `TransformerBlock` and `TokenAndPositionEmbedding`.

`TransformerBlock` is a custom layer that implements a single transformer block. This includes multi-head self-attention and a feed-forward neural network, each followed by layer normalization and dropout. The `call` method defines the forward pass of the transformer block.

`TokenAndPositionEmbedding` is another custom layer that creates embeddings for the tokens and adds positional encoding. Positional encoding is necessary because the self-attention mechanism in the transformer block doesn't have any inherent notion of position/order of the tokens in the sequence.

After defining these classes, this code block sets up the overall model architecture by creating instances of these layers and connecting them. The model takes as input a sequence of integers (the tokenized text), passes them through the embedding layer, then through the transformer block, and finally through a global average pooling layer and two dense layers. The output layer uses softmax activation to produce probabilities for the two classes (spam or not spam).

The model is compiled with the Adam optimizer and sparse categorical cross-entropy loss, which is suitable for multi-class classification where the labels are integers. It takes around 1 minute 30 seconds to complete.

Last Performance Metrics:
```
Precision: 0.993006993006993
Recall: 0.9403973509933775
F1 score: 0.9659863945578232
Accuracy: 0.9910314083099365
```

In [17]:
# Define the TransformerBlock as a layer
class TransformerBlock(layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super().__init__()
        self.att = layers.MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = keras.Sequential(
            [layers.Dense(ff_dim, activation="relu"), layers.Dense(embed_dim),]
        )
        self.layernorm1 = layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = layers.LayerNormalization(epsilon=1e-6)
        self.dropout1 = layers.Dropout(rate)
        self.dropout2 = layers.Dropout(rate)

    def call(self, inputs, training):
        attn_output = self.att(inputs, inputs)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

# Define the TokenAndPositionEmbedding layer    
class TokenAndPositionEmbedding(layers.Layer):
    def __init__(self, maxlen, vocab_size, embed_dim):
        super().__init__()
        self.token_emb = layers.Embedding(input_dim=vocab_size, output_dim=embed_dim)
        self.pos_emb = layers.Embedding(input_dim=maxlen, output_dim=embed_dim)

    def call(self, x):
        maxlen = tf.shape(x)[-1]
        positions = tf.range(start=0, limit=maxlen, delta=1)
        positions = self.pos_emb(positions)
        x = self.token_emb(x)
        return x + positions

# Embedding Size for each token
embed_dim = 32
# Number of attention heads
num_heads = 2
# Hidden layer size in feed forward network inside transformer
ff_dim = 32

# Define the model
inputs = layers.Input(shape=(max_length,))
embedding_layer = TokenAndPositionEmbedding(max_length, vocab_size, embed_dim)
x = embedding_layer(inputs)
transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim, rate=0.1)
x = transformer_block(x)
x = layers.GlobalAveragePooling1D()(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(20, activation="relu")(x)
x = layers.Dropout(0.1)(x)
outputs = layers.Dense(2, activation="softmax")(x)
model = keras.Model(inputs=inputs, outputs=outputs)
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])

# Train the model
history = model.fit(train_padded, train_data['label'], epochs=10, batch_size=32, validation_split=0.2)

# Evaluate the model
test_loss, test_acc = model.evaluate(test_padded, test_data['label'])

# Make predictions on the test data
y_pred = model.predict(test_padded)
y_pred = np.argmax(y_pred, axis=1)

# Calculate performance metrics
precision = precision_score(test_data['label'], y_pred)
recall = recall_score(test_data['label'], y_pred)
f1 = f1_score(test_data['label'], y_pred)

# Print the performance metrics
print("Transformer Results: ")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 score: {f1}")
print(f"Accuracy: {test_acc}")


Epoch 1/10
112/112 [==============================] - 9s 56ms/step - loss: 0.3721 - accuracy: 0.8648 - val_loss: 0.2901 - val_accuracy: 0.8688
Epoch 2/10
112/112 [==============================] - 6s 55ms/step - loss: 0.1817 - accuracy: 0.9327 - val_loss: 0.0451 - val_accuracy: 0.9865
Epoch 3/10
112/112 [==============================] - 6s 54ms/step - loss: 0.0509 - accuracy: 0.9851 - val_loss: 0.0385 - val_accuracy: 0.9877
Epoch 4/10
112/112 [==============================] - 6s 55ms/step - loss: 0.0209 - accuracy: 0.9935 - val_loss: 0.0392 - val_accuracy: 0.9877
Epoch 5/10
112/112 [==============================] - 6s 54ms/step - loss: 0.0200 - accuracy: 0.9930 - val_loss: 0.0402 - val_accuracy: 0.9888
Epoch 6/10
112/112 [==============================] - 6s 53ms/step - loss: 0.0068 - accuracy: 0.9989 - val_loss: 0.0478 - val_accuracy: 0.9877
Epoch 7/10
112/112 [==============================] - 6s 53ms/step - loss: 0.0015 - accuracy: 0.9994 - val_loss: 0.0678 - val_accuracy: 0.9888